### Milling Testing Notebook


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils
from fibsem import acquire
from fibsem.structures import ImageSettings, BeamType

import numpy as np
import matplotlib.pyplot as plt

PROTOCOL_PATH = r"C:\Users\Admin\Github\autolamella\autolamella\protocol\protocol-on-grid.yaml"

microscope, settings = utils.setup_session(protocol_path=PROTOCOL_PATH)

In [ ]:
images = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(images[0].data, cmap='gray')
ax[1].imshow(images[1].data, cmap='gray')
plt.show()

In [ ]:
from fibsem.milling import get_milling_stages
from fibsem.milling.patterning.plotting import generate_blank_image, draw_milling_patterns
stages = get_milling_stages("mill_rough", settings.protocol["milling"]) 
stages.extend(get_milling_stages("mill_polishing", settings.protocol["milling"]))
stages.extend(get_milling_stages("microexpansion", settings.protocol["milling"]))

settings.image.hfw = stages[0].milling.hfw
settings.image.beam_type = BeamType.ION
image = acquire.acquire_image(microscope, settings.image)

fig = draw_milling_patterns(image, stages)


In [ ]:
from fibsem.milling import mill_stages

mill_stages(microscope, stages)

In [3]:
stage = stages[1]
from fibsem.milling import mill_stages
# from fibsem.milling.strategy.overtilt import OvertiltTrenchMillingStrategy
# stage.strategy = OvertiltTrenchMillingStrategy(overtilt_deg=2)
print(stage.strategy.config)

mill_stages(microscope, [stage])


2024-12-05 15:49:54,777 — root — INFO — _set:2740 — ION current set to 3e-11 A.
2024-12-05 15:49:54,787 — root — INFO — _set:2744 — ION voltage set to 30000 V.
2024-12-05 15:49:54,800 — root — INFO — finish_milling:72 — Finished Ion Beam Milling.


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.milling import MillingStrategy
strat_d = {"name": "Overtilt", "overtilt": 2}
strat = MillingStrategy.from_dict(strat_d)
print(strat)
print(strat.config)

In [ ]:
strat.from_dict(strat.to_dict())

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.milling.base import get_strategy

strat = get_strategy(name="Overtilt", config={"config" :{"overtilt": 2}})
print(strat)
print(strat.config)

In [ ]:
from pprint import pprint
stage = stages[0]
stage.strategy = strat
ddict = stage.to_dict()

from fibsem.milling import FibsemMillingStage
stage = FibsemMillingStage.from_dict(ddict)
pprint(stage.strategy.config)